In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression

In [9]:
qb = pd.read_csv('../input/qb/dk_qb_stats-Copy1.csv')
week1 = pd.read_csv('../input/qb/week1_2018.csv')
week1_def = pd.read_csv('../input/defense_2018_projections.csv')

In [10]:
qb.head()


,Player,Team,Week,Opp,Comp,Att,Pct,Yds,Yds/Att,TD,Int,ru_att,ru_yds,yds/ru_att,ru_td,points,year,def_ru_rk,def_pass_rk
0,Matt Ryan,ATL,1,KC,23,31,74.0,299,10.0,3,0,3,25,8.0,1,32.46,2012,25,25
1,Robert Griffin III,WAS,1,NO,19,26,73.0,320,12.0,2,0,9,42,5.0,0,25.00,2012,31,31
2,Tony Romo,DAL,1,NYG,22,29,76.0,307,11.0,3,1,5,12,2.0,0,23.48,2012,12,12
3,Aaron Rodgers,GB,1,SF,30,44,68.0,303,7.0,2,1,5,27,5.0,0,22.82,2012,2,2
4,Drew Brees,NO,1,WAS,24,52,46.0,339,7.0,3,2,0,0,0.0,0,21.56,2012,22,22


## Current week dataframe feature engineering

In [11]:
week1.drop(['Rank','Position', 'PassingRating','FantasyPointsPerGame'], axis=1, inplace=True)

In [13]:
week1['year'] =2018
week1.columns = qb.columns[:-2]

In [14]:
indices = []
for idx,player in enumerate(week1['Player']):
    if player not in qb['Player'].unique():
        indices.append(idx)

week1.drop(indices, axis=0, inplace=True)
week1.reset_index(inplace=True, drop=True)
week1.shape

(57, 17)

## Weekly defensive rankings dataframe feature engineering

In [15]:
week1_def.index = week1_def['Team']
week1_def.drop('Team', axis=1, inplace=True)
week1_def.head()

,Rank,Name,Position,Played,TacklesForLoss,Sacks,QuarterbackHits,Interceptions,FumblesRecovered,Safeties,DefensiveTouchdowns,SpecialTeamsTouchdowns,PointsAllowed,FantasyPointsPerGameDraftKings,FantasyPointsDraftKings
Team,,,,,,,,,,,,,,,
JAX,1,Jacksonville Jaguars,DST,16,103,49,113,18,14,0,4,1,268,8.1,129
LAC,2,Los Angeles Chargers,DST,16,93,44,85,17,13,0,4,2,272,7.9,127
LAR,3,Los Angeles Rams,DST,16,82,45,100,17,15,2,3,2,329,7.8,124
PHI,4,Philadelphia Eagles,DST,16,99,39,115,17,14,0,4,2,295,7.7,123
MIN,5,Minnesota Vikings,DST,16,82,42,85,17,12,1,3,1,252,7.0,112


In [16]:
week1['def_ru_rk'] = list(map(lambda x: week1_def.loc[x]['Rank'], week1['Opp']))
week1['def_pass_rk'] = list(map(lambda x: week1_def.loc[x]['Rank'], week1['Opp']))

In [53]:
#A function that will covert a time series database into a supervised learning database

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
            cols.append(df.shift(i))
            names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [71]:
train  = qb
test = week1

In [72]:
qb.columns

Index(['Player', 'Team', 'Week', 'Opp', 'Comp', 'Att', 'Pct', 'Yds', 'Yds/Att',
       'TD', 'Int', 'ru_att', 'ru_yds', 'yds/ru_att', 'ru_td', 'points',
       'year', 'def_ru_rk', 'def_pass_rk'],
      dtype='object')

In [73]:
week1.columns

Index(['Player', 'Team', 'Week', 'Opp', 'Comp', 'Att', 'Pct', 'Yds', 'Yds/Att',
       'TD', 'Int', 'ru_att', 'ru_yds', 'yds/ru_att', 'ru_td', 'points',
       'year', 'def_ru_rk', 'def_pass_rk'],
      dtype='object')

In [345]:
passing = ['Comp','Att','Pct','Yds','Yds/Att','TD','Int']

In [74]:
y_train = train['points']
y_test = test['points']
X_train = train.drop('points',axis=1)
X_test = test.drop('points', axis=1)

In [75]:
X_train = X_train[X_train.columns[4:]]
X_test = X_test[X_test.columns[4:]]

In [348]:

X_train = X_train[X_train.columns[4:11]]
X_test = X_test[X_test.columns[4:11]]

In [76]:
X_train.shape

(2729, 14)

In [77]:
X_test.shape

(57, 14)

In [34]:
X_train.columns

Index(['Comp', 'Att', 'Pct', 'Yds', 'Yds/Att', 'TD', 'Int', 'ru_att', 'ru_yds',
       'yds/ru_att', 'ru_td', 'year', 'def_ru_rk', 'def_pass_rk'],
      dtype='object')

In [78]:
reframed = series_to_supervised(X_train.values.astype('float32'),1,1)

In [79]:
lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [80]:
coeff = pd.DataFrame(X_train.columns)
coeff['coefficients'] = lr.coef_
coeff



,0,coefficients
0,Comp,0.029938
1,Att,-0.076581
2,Pct,-0.013013
3,Yds,0.058688
4,Yds/Att,-0.179413
5,TD,4.002376
6,Int,-1.242807
7,ru_att,-0.086593
8,ru_yds,0.114284
9,yds/ru_att,-0.025317


In [81]:
yhat = lr.predict(X_test)
yhat.shape

(57,)

In [359]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

rmse_val = rmse(yhat, y_test)
print("RMSE error is: " + str(rmse_val))

RMSE error is: 1.68578446369


In [82]:
results = pd.DataFrame()
results['Player'] = week1['Player']
#results['score'] = y_test
results['Predicted'] = yhat
#results['Week'] = qb[qb['year']==2017]['Week']
#results.groupby('Player').mean()[['score','Predicted']]
results

,Player,Predicted
0,Aaron Rodgers,22.622505
1,Drew Brees,22.935390
2,Tom Brady,22.578469
3,Deshaun Watson,21.599166
4,Cam Newton,20.770403
5,Matthew Stafford,21.079952
6,Ben Roethlisberger,20.937474
7,Andrew Luck,20.630576
8,Philip Rivers,20.697301
9,Kirk Cousins,20.450940


In [323]:
results.shape

(300, 3)

In [84]:
results.to_csv('2018_week1_qb.csv', index= False)